# 第3回 その2: 時系列データの可視化 (2)：直線近似と曲線近似
時系列データの近似直線・近似曲線を書いてみましょう。  


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/03"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/03/
# 現在のフォルダの中身を表示
%ls

`energy.csv` と`covid19.csv`というデータが表示されていることを確認してください。

## ステップ1: 直線近似
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`energy.csv` を読み込み，データをプロットします。  
このデータは，1970年から2000年までの，日本の一世帯あたりの電力消費量を記録したデータです。  
(出典：一世帯あたりの電力消費量の推移 https://www.ene100.jp/zumen/1-2-13 の一部を抜粋)

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('energy.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) として抽出する。
year = csv_data.loc[:, '年'].to_numpy()
data = csv_data.loc[:, '電力消費量(kWh)'].to_numpy()

# 抽出したデータをプロット
plt.figure(figsize=(10,5))
plt.scatter(year, data, label='Real Data', color='b')
plt.xlabel('Year')
plt.ylabel('Power consumption [kWh]')
plt.legend()
plt.show()

このデータを線形関数 (y = ax + b)で近似します。  
近似関数の傾き(a)と切片(b)を計算する関数を以下に定義します。  

In [ ]:
def linear_approx(data):
  '''
      data: 線形直線で近似したいデータ
  '''
  N = np.size(data)
  y = np.array(data)
  x = np.arange(N) # x=[0, 1, ..., N-1]

  # 係数計算に必要な変数の計算
  xx = x * x
  xy = x * y
  
  # a と b の計算で共通する分母の計算
  bunbo = N * np.sum(xx) - np.sum(x)*np.sum(x)

  # a および b の計算
  a = (N*np.sum(xy) - np.sum(x)*np.sum(y)) / bunbo
  b = (np.sum(xx)*np.sum(y) - np.sum(xy)*np.sum(x)) / bunbo

  return (a, b)

上で定義した関数 `linear_approx` を使って，`data` の直線近似関数を計算します。

In [ ]:
# 直線近似関数のパラメータを求める
(a, b) = linear_approx(data)
print('approximate function: y = %fx + %f' % (a, b))

# 直線近似関数を作成
x = np.arange(np.size(data))
approx = a * x + b

近似結果によると，この期間おいては，毎月約 6kWhkずつ増加していることが分かります。   
実データと近似直線をプロットします。

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(year, data, label='Real Data', color='b')
plt.plot(year, approx, label='Approximate function', color='r')
plt.xlabel('Year')
plt.ylabel('Power consumption [kWh]')
plt.legend()
plt.show()

近似直線が実際の値に対してどの程度ずれているのか，計算してみましょう。

In [ ]:
# 平均二乗誤差(mean square error)
mse = np.mean((data - approx)*(data - approx))
# 平均平方根二乗誤差 (root mean square error) = mse の平方根
rmse = np.sqrt(mse)

print('MSE = %f' % (mse))
print('RMSE = %f' % (rmse))

平均で約 7.06 kWh 程度の誤差であることから，そこそこの精度で近似できているように見えます。

## ステップ2: 曲線近似
次に，曲線で近似することを考えます。  
曲線には色々種類が考えられますが（例：2次曲線，3次曲線，n次曲線，対数曲線，等），ここでは指数関数(y = b*e^{a*x})での近似を考えます。

`covid19.csv` を読み込み，データをプロットします。  
このデータは，2020年3月1日から2020年3月30日までの，全世界のCOVID-19の新規感染者数の記録です。  
(出典：Our World in Data https://ourworldindata.org/coronavirus-data )

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('covid19.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) として抽出する。
data = csv_data.loc[:, '新規感染者数(全世界)'].to_numpy()

# 抽出したデータをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(data))+1
plt.scatter(x, data, label='Real Data', color='b')
plt.xlabel('Date (2020/3/xx)')
plt.ylabel('Daily confirmed COVID-19 case')
plt.legend()
plt.show()

まずは，ステップ1と同様に直線で近似してみます。

In [ ]:
# 直線近似関数を求める
(a, b) = linear_approx(data)
x = np.arange(np.size(data))
approx_linear = a * x + b

# 直線近似関数をプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(data))+1
plt.scatter(x, data, label='Real Data', color='b')
plt.plot(x, approx_linear, label='Approximate function', color='r')
plt.xlabel('Date (2020/3/xx)')
plt.ylabel('Daily confirmed COVID-19 case')
plt.legend()
plt.show()

# 近似誤差を計算する
mse = np.mean((data - approx_linear)*(data - approx_linear))
rmse = np.sqrt(mse)
print('MSE = %f' % (mse))
print('RMSE = %f' % (rmse))

誤差が大きい上に，感染者数がマイナスの日も存在してしまっていることから，  
直線での近似がふさわしくないことが分かります。

次に，指数関数を用いて近似します。  
データを指数関数で近似したい場合は，データを対数変換することで，線形近似を求める定式化に当てはめることが出来ます。

In [ ]:
# データを対数変換する (numpy の log 関数を使用する)
log_data = np.log(data)

# 対数変換したデータに対して，線形近似関数のパラメータを求める
(a, log_b) = linear_approx(log_data)
# log_b を指数変換する
b = np.exp(log_b)

# 近似関数を作成
x = np.arange(np.size(data))
approx = b * np.exp(a * x)

実データと近似関数をプロットします。  
また，近似誤差も求めます。

In [ ]:
plt.figure(figsize=(10,5))
x = np.arange(np.size(data))+1
plt.scatter(x, data, label='Real Data', color='b')
plt.plot(x, approx, label='Approximate function', color='r')
plt.xlabel('Date (2020/3/xx)')
plt.ylabel('Daily confirmed COVID-19 case')
plt.legend()
plt.show()

# 平均二乗誤差(mean square error)
mse = np.mean((data - approx)*(data - approx))
# 平均平方根二乗誤差 (root mean square error) = mse の平方根
rmse = np.sqrt(mse)

print('MSE = %f' % (mse))
print('RMSE = %f' % (rmse))

概ねうまく近似できているように見えますが，実は29日，30日あたりは誤差が大きいため，誤差の平均値はそれほど小さくありません。

3月初めから3月後半までは指数的に増加しているように見えますが，  
実際は3月末辺りから新規感染者の増加は一旦緩やかになり，それまでの指数関数的増加から外れます。

試しに，25日までの誤差と26日以降の誤差を比較してみましょう。  
25日までと26日以降で誤差が大きく異なることが分かります。

In [ ]:
# 25日までの誤差
mse = np.mean((data[:25] - approx[:25])*(data[:25] - approx[:25]))
rmse = np.sqrt(mse)
print('MSE(3/1-25) = %f' % (mse))
print('RMSE(3/1-25) = %f' % (rmse))

# 26日以降の誤差
mse = np.mean((data[25:] - approx[25:])*(data[25:] - approx[25:]))
rmse = np.sqrt(mse)
print('MSE(3/26-30) = %f' % (mse))
print('RMSE(3/26-30) = %f' % (rmse))

ウイルス感染のメカニズム自体は指数的に増加する形になりますが，それは人口が無限に存在し，かつ人間が何の対策も取らなかった場合です。  
実際は，感染者が増えて健康な人の数が減ればその分感染のスピードは緩やかになりますし，当然人間側は感染者の隔離などの対策は取るわけなので，指数的増加現象は一部の期間に限定されます。

<font color="Red">この演習では，あくまで指数関数による近似の理論を説明することを目的として，その例として指数関数に当てはまっている期間を限定的に抽出し，解析を行ったに過ぎません。本講義はコロナウイルスの問題に対して何らかの意図や見解を含むものではありませんので，誤解の無いようにお願いします。</font>